# Update Metadata với Status Enums Mới

Notebook này cập nhật metadata cho các documents đã có trong database theo schema mới:
- **DocumentStatus**: Trạng thái hiệu lực của tài liệu (ACTIVE, OUTDATED, SUPERSEDED, etc.)
- **ProcessingStatus**: Trạng thái xử lý pipeline (PENDING, COMPLETED, FAILED, etc.)

## Cấu trúc Status 3 tầng:
1. **DocumentStatus** (Hiệu lực tài liệu) - Áp dụng cho TẤT CẢ loại documents
   - ACTIVE: Đang sử dụng
   - OUTDATED: Đã lỗi thời
   - SUPERSEDED: Bị thay thế bởi doc khác
   - EXPIRED: Hết hạn
   - ARCHIVED: Đã lưu trữ
   
2. **LegalStatus** (Hiệu lực pháp lý) - CHỈ áp dụng cho văn bản pháp luật
   - CON_HIEU_LUC: Còn hiệu lực
   - HET_HIEU_LUC: Hết hiệu lực
   - BI_THAY_THE: Bị thay thế
   
3. **ProcessingStatus** (Trạng thái xử lý) - Áp dụng cho TẤT CẢ documents
   - PENDING: Chưa xử lý
   - IN_PROGRESS: Đang xử lý
   - COMPLETED: Hoàn thành
   - FAILED: Thất bại

## 1. Import Libraries và Setup

In [1]:
# No need to install - already in environment
# psycopg2 is for sync connection to PostgreSQL
import psycopg2
from psycopg2.extras import Json
import json
from datetime import datetime
from typing import Dict, List, Any

print("✅ psycopg2 imported successfully")


✅ psycopg2 imported successfully


In [2]:
# Add project root to Python path
import sys
from pathlib import Path

# Get project root (parent of notebooks folder)
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✅ Added {project_root} to Python path")


✅ Added /home/sakana/Code/RAG-bidding to Python path


In [3]:
# Import settings and enums from project
from src.config.models import settings
from src.preprocessing.schema.enums import DocumentStatus, ProcessingStatus

print("✅ Project modules imported successfully")
print(f"\nDocumentStatus values: {[s.value for s in DocumentStatus]}")
print(f"ProcessingStatus values: {[s.value for s in ProcessingStatus]}")


✅ Project modules imported successfully

DocumentStatus values: ['active', 'draft', 'outdated', 'superseded', 'expired', 'archived', 'deprecated', 'under_revision']
ProcessingStatus values: ['pending', 'in_progress', 'completed', 'failed', 'partial', 'skipped', 'retry']


## 2. Kết nối Database và Kiểm tra Schema

In [4]:
# Connect to database using psycopg2 (synchronous connection)
# Convert async SQLAlchemy URL to standard PostgreSQL DSN
dsn = settings.database_url.replace("postgresql+psycopg", "postgresql")

print(f"Connecting to database...")
print(f"DSN (password hidden): {dsn.split('@')[1] if '@' in dsn else 'local'}")

try:
    conn = psycopg2.connect(dsn)
    conn.autocommit = False  # We want explicit transaction control
    print("✅ Database connection established successfully!")
    
    # Test connection with a simple query
    with conn.cursor() as cur:
        cur.execute("SELECT version();")
        version = cur.fetchone()[0]
        print(f"\nPostgreSQL version: {version[:50]}...")
        
        # Check if langchain tables exist
        cur.execute("""
            SELECT COUNT(*) 
            FROM information_schema.tables 
            WHERE table_name IN ('langchain_pg_collection', 'langchain_pg_embedding')
        """)
        table_count = cur.fetchone()[0]
        print(f"Found {table_count}/2 langchain tables")
        
except Exception as e:
    print(f"❌ Connection failed: {e}")
    raise


Connecting to database...
DSN (password hidden): localhost:5432/rag_bidding_v2
✅ Database connection established successfully!

PostgreSQL version: PostgreSQL 18.0 (Ubuntu 18.0-1.pgdg24.04+3) on x86...
Found 2/2 langchain tables


## 3. Xem Sample Metadata hiện tại

In [5]:
# Get collection UUID first
with conn.cursor() as cur:
    cur.execute("""
        SELECT uuid, name 
        FROM langchain_pg_collection 
        LIMIT 5;
    """)
    collections = cur.fetchall()
    
    print("📦 Available collections:")
    for coll_uuid, coll_name in collections:
        print(f"   - {coll_name}: {coll_uuid}")
    
    # Get the main collection (usually first one)
    collection_uuid = collections[0][0] if collections else None
    collection_name = collections[0][1] if collections else None
    print(f"\n✅ Using collection: {collection_name} ({collection_uuid})")

# Count total embeddings in this collection
with conn.cursor() as cur:
    cur.execute("""
        SELECT COUNT(*) 
        FROM langchain_pg_embedding 
        WHERE collection_id = %s;
    """, (collection_uuid,))
    
    total_count = cur.fetchone()[0]
    print(f"\n📊 Total documents in collection: {total_count}")

# Get 3 sample documents
with conn.cursor() as cur:
    cur.execute("""
        SELECT 
            id,
            cmetadata->>'doc_type' as doc_type,
            cmetadata->>'doc_id' as doc_id,
            cmetadata->>'title' as title,
            cmetadata
        FROM langchain_pg_embedding
        WHERE collection_id = %s
        LIMIT 3;
    """, (collection_uuid,))
    
    samples = cur.fetchall()
    print("\n📄 Sample documents and their current metadata:")
    print("=" * 80)
    
    for idx, (doc_id, doc_type, chunk_doc_id, title, metadata) in enumerate(samples, 1):
        print(f"\n{idx}. Document ID: {doc_id}")
        print(f"   Doc Type: {doc_type}")
        print(f"   Chunk Doc ID: {chunk_doc_id}")
        print(f"   Title: {title}")
        print(f"   Metadata keys: {list(metadata.keys())}")
        print(f"   Full metadata:")
        print(f"   {json.dumps(metadata, indent=6, ensure_ascii=False)}")


📦 Available collections:
   - docs: 404592e1-5956-41d1-b736-cee216941e37

✅ Using collection: docs (404592e1-5956-41d1-b736-cee216941e37)

📊 Total documents in collection: 4708

📄 Sample documents and their current metadata:

1. Document ID: 81c3735e-aa95-4f5a-ba52-ce10ca6b36fc
   Doc Type: None
   Chunk Doc ID: None
   Title: None
   Metadata keys: ['level', 'chunk_id', 'has_list', 'has_table', 'hierarchy', 'char_count', 'chunk_index', 'document_id', 'total_chunks', 'document_info', 'document_type', 'section_title', 'extra_metadata', 'is_complete_unit', 'processing_metadata']
   Full metadata:
   {
      "level": "dieu",
      "chunk_id": "circular_untitled_dieu_0001",
      "has_list": true,
      "has_table": false,
      "hierarchy": "[\"Điều 2. Đối tượng áp dụng\"]",
      "char_count": 451,
      "chunk_index": 1,
      "document_id": "circular_untitled",
      "total_chunks": 104,
      "document_info": {
            "document_status": "active"
      },
      "document_type": "c

## 4. Chuẩn bị Update Functions

Tạo các helper functions để update metadata với status fields mới

In [6]:
def add_document_status_to_metadata(metadata: Dict[str, Any], status: DocumentStatus = DocumentStatus.ACTIVE) -> Dict[str, Any]:
    """
    Thêm document_status vào metadata.
    Mặc định là ACTIVE cho tất cả documents hiện có.
    """
    # Tạo copy để không modify original
    updated = metadata.copy()
    
    # Thêm document_status vào document_info
    if 'document_info' in updated:
        updated['document_info']['document_status'] = status.value
    else:
        # Nếu chưa có document_info, tạo mới
        updated['document_info'] = {'document_status': status.value}
    
    return updated


def add_processing_status_to_metadata(metadata: Dict[str, Any], 
                                      status: ProcessingStatus = ProcessingStatus.COMPLETED) -> Dict[str, Any]:
    """
    Thêm processing_status vào processing_metadata.
    Mặc định là COMPLETED vì các documents đã được process thành công.
    """
    updated = metadata.copy()
    
    # Thêm/update processing_metadata
    if 'processing_metadata' not in updated:
        updated['processing_metadata'] = {}
    
    updated['processing_metadata']['processing_status'] = status.value
    updated['processing_metadata']['last_processed_at'] = datetime.now().isoformat()
    
    # Nếu COMPLETED thì không có error
    if status == ProcessingStatus.COMPLETED:
        updated['processing_metadata']['error_message'] = None
        updated['processing_metadata']['retry_count'] = 0
    
    return updated


def update_document_metadata_batch(updates: List[tuple], conn, commit: bool = True) -> int:
    """
    Batch update nhiều documents cùng lúc.
    Returns số lượng documents updated thành công.
    
    Args:
        updates: List of (doc_uuid, new_metadata) tuples
        conn: psycopg2 connection object
        commit: Có commit ngay không (default True)
    """
    try:
        with conn.cursor() as cur:
            for doc_uuid, new_metadata in updates:
                cur.execute("""
                    UPDATE langchain_pg_embedding
                    SET cmetadata = %s
                    WHERE id = %s;
                """, (Json(new_metadata), doc_uuid))
        
        if commit:
            conn.commit()
        
        return len(updates)
    except Exception as e:
        print(f"❌ Batch update error: {e}")
        conn.rollback()
        return 0


print("✅ Helper functions defined:")
print("  - add_document_status_to_metadata()")
print("  - add_processing_status_to_metadata()")
print("  - update_document_metadata_batch() [NEW - faster!]")


✅ Helper functions defined:
  - add_document_status_to_metadata()
  - add_processing_status_to_metadata()
  - update_document_metadata_batch() [NEW - faster!]


## 5. Test Update trên 1 Document

Thử update một document để verify logic hoạt động đúng

In [12]:
# Lấy 1 document để test update thật
with conn.cursor() as cur:
    cur.execute("""
        SELECT id, cmetadata 
        FROM langchain_pg_embedding 
        WHERE collection_id = %s
        LIMIT 1;
    """, (collection_uuid,))
    test_id, test_metadata = cur.fetchone()

print(f"🧪 Testing on document ID: {test_id}")
print(f"\n📋 Original metadata keys: {list(test_metadata.keys())}")
print(f"   Has document_info? {'document_info' in test_metadata}")
print(f"   Has processing_metadata? {'processing_metadata' in test_metadata}")

# Apply updates
updated_metadata = add_document_status_to_metadata(test_metadata, DocumentStatus.ACTIVE)
updated_metadata = add_processing_status_to_metadata(updated_metadata, ProcessingStatus.COMPLETED)

print(f"\n📝 Updated metadata:")
print(f"   New keys: {set(updated_metadata.keys()) - set(test_metadata.keys())}")
print(f"   document_info: {updated_metadata.get('document_info', {})}")
print(f"   processing_metadata keys: {list(updated_metadata.get('processing_metadata', {}).keys())}")

# Update in database
print(f"\n🔄 Updating database...")
success = update_document_metadata(test_id, updated_metadata, conn)

if success:
    print(f"✅ Successfully updated document!")
    
    # Verify update
    with conn.cursor() as cur:
        cur.execute("""
            SELECT cmetadata 
            FROM langchain_pg_embedding 
            WHERE id = %s;
        """, (test_id,))
        verified_metadata = cur.fetchone()[0]
    
    doc_status = verified_metadata.get('document_info', {}).get('document_status')
    proc_status = verified_metadata.get('processing_metadata', {}).get('processing_status')
    
    print(f"\n🔍 Verification from database:")
    print(f"   document_status: {doc_status}")
    print(f"   processing_status: {proc_status}")
    print(f"   Has all expected keys? {all(k in verified_metadata for k in ['document_info', 'processing_metadata'])}")
else:
    print(f"❌ Failed to update document {test_id}")


🧪 Testing on document ID: e431bb39-31a5-4850-be21-c7427d170f9f

📋 Original metadata keys: ['level', 'chunk_id', 'has_list', 'has_table', 'hierarchy', 'char_count', 'chunk_index', 'document_id', 'total_chunks', 'document_type', 'section_title', 'extra_metadata', 'is_complete_unit']
   Has document_info? False
   Has processing_metadata? False

📝 Updated metadata:
   New keys: {'document_info', 'processing_metadata'}
   document_info: {'document_status': 'active'}
   processing_metadata keys: ['processing_status', 'last_processed_at', 'error_message', 'retry_count']

🔄 Updating database...
✅ Successfully updated document!

🔍 Verification from database:
   document_status: active
   processing_status: completed
   Has all expected keys? True


## 6. Batch Update TẤT CẢ Documents

Update tất cả documents trong database với status mặc định:
- **document_status** = ACTIVE (vì tất cả documents hiện tại đều đang được sử dụng)
- **processing_status** = COMPLETED (vì đã được process thành công)

⚠️ **Lưu ý**: Chạy cell này sẽ update TẤT CẢ documents trong database!

In [8]:
# ⚠️ BATCH UPDATE - OPTIMIZED VERSION
# Update TẤT CẢ documents với batch commit để tăng tốc
COMMIT_BATCH_SIZE = 500  # Commit mỗi 500 documents

print(f"⚙️  OPTIMIZED BATCH UPDATE CONFIGURATION:")
print(f"   • Total documents in DB: {total_count}")
print(f"   • Commit batch size: {COMMIT_BATCH_SIZE} (commits mỗi {COMMIT_BATCH_SIZE} docs)")

# Đếm số documents CHƯA update
with conn.cursor() as cur:
    cur.execute("""
        SELECT COUNT(*) 
        FROM langchain_pg_embedding
        WHERE collection_id = %s
          AND cmetadata->'document_info'->>'document_status' IS NULL;
    """, (collection_uuid,))
    docs_to_update = cur.fetchone()[0]

print(f"   • Documents cần update: {docs_to_update}")
print(f"   • Estimated time: ~{docs_to_update / 1000:.1f} seconds")
print("=" * 80)

# Lấy ALL documents CHƯA update
with conn.cursor() as cur:
    cur.execute("""
        SELECT id, cmetadata 
        FROM langchain_pg_embedding 
        WHERE collection_id = %s
          AND cmetadata->'document_info'->>'document_status' IS NULL;
    """, (collection_uuid,))
    batch_docs = cur.fetchall()

print(f"\n🚀 Starting OPTIMIZED batch update for {len(batch_docs)} documents...")
print("-" * 80)

success_count = 0
failed_count = 0
start_time = datetime.now()

# Process in batches for commit
pending_updates = []

for i, (doc_id, metadata) in enumerate(batch_docs, 1):
    # Apply updates
    updated = add_document_status_to_metadata(metadata, DocumentStatus.ACTIVE)
    updated = add_processing_status_to_metadata(updated, ProcessingStatus.COMPLETED)
    
    # Add to pending batch
    pending_updates.append((doc_id, updated))
    
    # Commit batch khi đủ số lượng hoặc cuối cùng
    if len(pending_updates) >= COMMIT_BATCH_SIZE or i == len(batch_docs):
        updated_count = update_document_metadata_batch(pending_updates, conn, commit=True)
        success_count += updated_count
        failed_count += len(pending_updates) - updated_count
        pending_updates = []  # Reset batch
        
        # Progress indicator
        elapsed = (datetime.now() - start_time).total_seconds()
        rate = i / elapsed if elapsed > 0 else 0
        remaining = len(batch_docs) - i
        eta = remaining / rate if rate > 0 else 0
        print(f"Progress: {i}/{len(batch_docs)} ({i/len(batch_docs)*100:.1f}%) - "
              f"{rate:.1f} docs/sec - ✅ {success_count} ❌ {failed_count} - ETA: {eta:.1f}s")

end_time = datetime.now()
duration = (end_time - start_time).total_seconds()

print("\n" + "=" * 80)
print("📈 Batch Update Summary:")
print(f"  Total processed: {len(batch_docs)}")
print(f"  ✅ Successfully updated: {success_count}")
print(f"  ❌ Failed: {failed_count}")
print(f"  ⏱️  Duration: {duration:.2f} seconds")
print(f"  📊 Average rate: {len(batch_docs)/duration:.1f} docs/sec")

# Re-count updated documents
with conn.cursor() as cur:
    cur.execute("""
        SELECT COUNT(*) 
        FROM langchain_pg_embedding
        WHERE collection_id = %s
          AND cmetadata->'document_info'->>'document_status' IS NOT NULL;
    """, (collection_uuid,))
    updated_total = cur.fetchone()[0]

print(f"\n📊 Database Status:")
print(f"  • Total documents with status: {updated_total}/{total_count} ({updated_total/total_count*100:.1f}%)")
print(f"  • Remaining to update: {total_count - updated_total}")

if success_count == len(batch_docs):
    print(f"\n🎉 Batch completed successfully!")
    if updated_total >= total_count:
        print("✅ ALL DOCUMENTS UPDATED! Database migration complete!")
    else:
        print(f"⚠️  Still {total_count - updated_total} documents to update. Re-run this cell.")
else:
    print(f"\n⚠️  Some updates failed. Check errors above.")


⚙️  OPTIMIZED BATCH UPDATE CONFIGURATION:
   • Total documents in DB: 4708
   • Commit batch size: 500 (commits mỗi 500 docs)
   • Documents cần update: 4697
   • Estimated time: ~4.7 seconds

🚀 Starting OPTIMIZED batch update for 4697 documents...
--------------------------------------------------------------------------------
Progress: 500/4697 (10.6%) - 7577.1 docs/sec - ✅ 500 ❌ 0 - ETA: 0.6s
Progress: 1000/4697 (21.3%) - 7125.7 docs/sec - ✅ 1000 ❌ 0 - ETA: 0.5s
Progress: 1500/4697 (31.9%) - 6581.4 docs/sec - ✅ 1500 ❌ 0 - ETA: 0.5s
Progress: 2000/4697 (42.6%) - 6463.1 docs/sec - ✅ 2000 ❌ 0 - ETA: 0.4s
Progress: 1000/4697 (21.3%) - 7125.7 docs/sec - ✅ 1000 ❌ 0 - ETA: 0.5s
Progress: 1500/4697 (31.9%) - 6581.4 docs/sec - ✅ 1500 ❌ 0 - ETA: 0.5s
Progress: 2000/4697 (42.6%) - 6463.1 docs/sec - ✅ 2000 ❌ 0 - ETA: 0.4s
Progress: 2500/4697 (53.2%) - 6320.7 docs/sec - ✅ 2500 ❌ 0 - ETA: 0.3s
Progress: 3000/4697 (63.9%) - 6220.1 docs/sec - ✅ 3000 ❌ 0 - ETA: 0.3s
Progress: 3500/4697 (74.5%) - 637

## 7. Verification - Kiểm tra kết quả

Verify rằng tất cả documents đã được update đúng

In [9]:
# Verification - Kiểm tra kết quả update
print("🔍 Verification Results:")
print("=" * 80)

# 1. Kiểm tra số lượng documents có document_status
with conn.cursor() as cur:
    cur.execute("""
        SELECT COUNT(*) 
        FROM langchain_pg_embedding
        WHERE collection_id = %s
          AND cmetadata->'document_info'->>'document_status' IS NOT NULL;
    """, (collection_uuid,))
    with_doc_status = cur.fetchone()[0]

# 2. Kiểm tra số lượng documents có processing_status  
with conn.cursor() as cur:
    cur.execute("""
        SELECT COUNT(*) 
        FROM langchain_pg_embedding
        WHERE collection_id = %s
          AND cmetadata->'processing_metadata'->>'processing_status' IS NOT NULL;
    """, (collection_uuid,))
    with_proc_status = cur.fetchone()[0]

# 3. Kiểm tra phân bố document_status
with conn.cursor() as cur:
    cur.execute("""
        SELECT 
            cmetadata->'document_info'->>'document_status' as doc_status,
            COUNT(*) as count
        FROM langchain_pg_embedding
        WHERE collection_id = %s
          AND cmetadata->'document_info'->>'document_status' IS NOT NULL
        GROUP BY doc_status
        ORDER BY count DESC;
    """, (collection_uuid,))
    doc_status_dist = cur.fetchall()

# 4. Kiểm tra phân bố processing_status
with conn.cursor() as cur:
    cur.execute("""
        SELECT 
            cmetadata->'processing_metadata'->>'processing_status' as proc_status,
            COUNT(*) as count
        FROM langchain_pg_embedding
        WHERE collection_id = %s
          AND cmetadata->'processing_metadata'->>'processing_status' IS NOT NULL
        GROUP BY proc_status
        ORDER BY count DESC;
    """, (collection_uuid,))
    proc_status_dist = cur.fetchall()

# 5. Lấy sample để verify
with conn.cursor() as cur:
    cur.execute("""
        SELECT 
            cmetadata->>'document_type' as doc_type,
            cmetadata->>'document_id' as doc_id,
            cmetadata->'document_info'->>'document_status' as doc_status,
            cmetadata->'processing_metadata'->>'processing_status' as proc_status,
            cmetadata->'processing_metadata'->>'last_processed_at' as last_processed
        FROM langchain_pg_embedding
        WHERE collection_id = %s
          AND cmetadata->'document_info'->>'document_status' IS NOT NULL
        LIMIT 5;
    """, (collection_uuid,))
    samples = cur.fetchall()

print(f"\n1️⃣  Documents with document_status: {with_doc_status}/{total_count}")
print(f"2️⃣  Documents with processing_status: {with_proc_status}/{total_count}")

print(f"\n3️⃣  DocumentStatus distribution:")
for status, count in doc_status_dist:
    print(f"    {status:20} {count:>6} docs")

print(f"\n4️⃣  ProcessingStatus distribution:")
for status, count in proc_status_dist:
    print(f"    {status:20} {count:>6} docs")

print(f"\n5️⃣  Sample updated documents:")
print("-" * 80)
for doc_type, doc_id, doc_status, proc_status, last_processed in samples:
    print(f"  Type: {doc_type:10} | Doc ID: {doc_id:30}")
    print(f"         DocumentStatus: {doc_status:15} | ProcessingStatus: {proc_status}")
    print(f"         Last processed: {last_processed}")
    print()


🔍 Verification Results:

1️⃣  Documents with document_status: 4708/4708
2️⃣  Documents with processing_status: 4708/4708

3️⃣  DocumentStatus distribution:
    active                 4708 docs

4️⃣  ProcessingStatus distribution:
    completed              4708 docs

5️⃣  Sample updated documents:
--------------------------------------------------------------------------------
  Type: circular   | Doc ID: circular_untitled             
         DocumentStatus: active          | ProcessingStatus: completed
         Last processed: 2025-11-09T14:06:20.860476

  Type: circular   | Doc ID: circular_untitled             
         DocumentStatus: active          | ProcessingStatus: completed
         Last processed: 2025-11-09T14:06:20.873220

  Type: bidding    | Doc ID: bidding_untitled              
         DocumentStatus: active          | ProcessingStatus: completed
         Last processed: 2025-11-09T14:06:20.876510

  Type: circular   | Doc ID: circular_untitled             
        

### 📋 Sample Document - Xem Chi Tiết Kết Quả

Lấy 1 document đã update để xem toàn bộ metadata structure

In [10]:
# Lấy 1 sample document ĐÃ UPDATE để xem chi tiết
print("🔍 SAMPLE DOCUMENT - KẾT QUẢ SAU KHI UPDATE")
print("=" * 100)

with conn.cursor() as cur:
    cur.execute("""
        SELECT 
            id,
            cmetadata
        FROM langchain_pg_embedding
        WHERE collection_id = %s
          AND cmetadata->'document_info'->>'document_status' IS NOT NULL
        LIMIT 1;
    """, (collection_uuid,))
    
    sample_id, sample_metadata = cur.fetchone()

print(f"\n📌 Document UUID: {sample_id}")
print(f"\n📊 METADATA STRUCTURE:")
print("-" * 100)

# In ra metadata với format đẹp
print(json.dumps(sample_metadata, indent=2, ensure_ascii=False))

print("\n" + "=" * 100)
print("🎯 ĐIỂM QUAN TRỌNG CẦN KIỂM TRA:")
print("=" * 100)

# Highlight các fields quan trọng
print(f"\n1️⃣  DOCUMENT INFO:")
doc_info = sample_metadata.get('document_info', {})
for key, value in doc_info.items():
    print(f"   ✓ {key}: {value}")

print(f"\n2️⃣  PROCESSING METADATA:")
proc_metadata = sample_metadata.get('processing_metadata', {})
for key, value in proc_metadata.items():
    print(f"   ✓ {key}: {value}")

print(f"\n3️⃣  ORIGINAL METADATA FIELDS (vẫn giữ nguyên):")
original_fields = ['document_type', 'document_id', 'chunk_id', 'level', 'section_title']
for field in original_fields:
    if field in sample_metadata:
        print(f"   ✓ {field}: {sample_metadata[field]}")

print(f"\n4️⃣  TOTAL METADATA KEYS: {len(sample_metadata)}")
print(f"   All keys: {list(sample_metadata.keys())}")

print("\n" + "=" * 100)
print("💡 ĐÁNH GIÁ:")
print("   ✅ Nếu bạn thấy 'document_info' và 'processing_metadata' xuất hiện")
print("   ✅ Và các fields cũ vẫn còn nguyên")
print("   ✅ Thì có thể APPLY lên toàn bộ 4708 documents!")
print("=" * 100)

🔍 SAMPLE DOCUMENT - KẾT QUẢ SAU KHI UPDATE

📌 Document UUID: 81c3735e-aa95-4f5a-ba52-ce10ca6b36fc

📊 METADATA STRUCTURE:
----------------------------------------------------------------------------------------------------
{
  "level": "dieu",
  "chunk_id": "circular_untitled_dieu_0001",
  "has_list": true,
  "has_table": false,
  "hierarchy": "[\"Điều 2. Đối tượng áp dụng\"]",
  "char_count": 451,
  "chunk_index": 1,
  "document_id": "circular_untitled",
  "total_chunks": 104,
  "document_info": {
    "document_status": "active"
  },
  "document_type": "circular",
  "section_title": "Đối tượng áp dụng",
  "extra_metadata": "{\"dieu_number\": \"2\", \"khoan_number\": null, \"phan\": null, \"chuong\": null, \"muc\": null, \"entities\": {\"laws\": [], \"decrees\": [], \"circulars\": [], \"decisions\": [], \"dates\": [], \"organizations\": []}, \"referenced_laws\": [], \"referenced_decrees\": [], \"referenced_circulars\": [], \"referenced_dates\": [], \"organizations\": [], \"concepts\": [

### 🔄 So Sánh BEFORE vs AFTER

Xem sự khác biệt giữa document chưa update và đã update

In [16]:
# So sánh BEFORE (chưa update) vs AFTER (đã update)
print("🔄 SO SÁNH: BEFORE UPDATE vs AFTER UPDATE")
print("=" * 100)

# Lấy 1 doc CHƯA update
with conn.cursor() as cur:
    cur.execute("""
        SELECT cmetadata
        FROM langchain_pg_embedding
        WHERE collection_id = %s
          AND cmetadata->'document_info'->>'document_status' IS NULL
        LIMIT 1;
    """, (collection_uuid,))
    
    result = cur.fetchone()
    before_metadata = result[0] if result else None

# Lấy 1 doc ĐÃ update
with conn.cursor() as cur:
    cur.execute("""
        SELECT cmetadata
        FROM langchain_pg_embedding
        WHERE collection_id = %s
          AND cmetadata->'document_info'->>'document_status' IS NOT NULL
        LIMIT 1;
    """, (collection_uuid,))
    
    after_metadata = cur.fetchone()[0]

if before_metadata:
    print("\n📋 BEFORE UPDATE (Document chưa có status):")
    print("-" * 100)
    print(f"   Total keys: {len(before_metadata)}")
    print(f"   Keys: {list(before_metadata.keys())}")
    print(f"   ❌ Has 'document_info'? {('document_info' in before_metadata)}")
    print(f"   ❌ Has 'processing_metadata'? {('processing_metadata' in before_metadata)}")
else:
    print("\n⚠️  Tất cả documents đã được update rồi! Không còn document nào chưa có status.")

print("\n📋 AFTER UPDATE (Document đã có status):")
print("-" * 100)
print(f"   Total keys: {len(after_metadata)}")
print(f"   Keys: {list(after_metadata.keys())}")
print(f"   ✅ Has 'document_info'? {('document_info' in after_metadata)}")
print(f"   ✅ Has 'processing_metadata'? {('processing_metadata' in after_metadata)}")

if 'document_info' in after_metadata:
    print(f"\n   document_info content:")
    for k, v in after_metadata['document_info'].items():
        print(f"      • {k}: {v}")

if 'processing_metadata' in after_metadata:
    print(f"\n   processing_metadata content:")
    for k, v in after_metadata['processing_metadata'].items():
        print(f"      • {k}: {v}")

print("\n" + "=" * 100)
print("📊 THỐNG KÊ:")
print(f"   • Documents ĐÃ UPDATE: {with_doc_status}/{total_count} ({with_doc_status/total_count*100:.1f}%)")
print(f"   • Documents CHƯA UPDATE: {total_count - with_doc_status}/{total_count} ({(total_count - with_doc_status)/total_count*100:.1f}%)")
print("=" * 100)

print("\n✅ KẾT LUẬN:")
print("   1. ✅ Metadata CŨ được giữ nguyên 100%")
print("   2. ✅ Thêm được 2 sections MỚI: 'document_info' và 'processing_metadata'")
print("   3. ✅ Không làm mất hoặc thay đổi bất kỳ field nào")
print("   4. ✅ Structure hoàn toàn tương thích với schema mới")
print(f"   5. 💡 Sẵn sàng update {total_count - with_doc_status} documents còn lại!")


🔄 SO SÁNH: BEFORE UPDATE vs AFTER UPDATE

📋 BEFORE UPDATE (Document chưa có status):
----------------------------------------------------------------------------------------------------
   Total keys: 13
   Keys: ['level', 'chunk_id', 'has_list', 'has_table', 'hierarchy', 'char_count', 'chunk_index', 'document_id', 'total_chunks', 'document_type', 'section_title', 'extra_metadata', 'is_complete_unit']
   ❌ Has 'document_info'? False
   ❌ Has 'processing_metadata'? False

📋 AFTER UPDATE (Document đã có status):
----------------------------------------------------------------------------------------------------
   Total keys: 15
   Keys: ['level', 'chunk_id', 'has_list', 'has_table', 'hierarchy', 'char_count', 'chunk_index', 'document_id', 'total_chunks', 'document_info', 'document_type', 'section_title', 'extra_metadata', 'is_complete_unit', 'processing_metadata']
   ✅ Has 'document_info'? True
   ✅ Has 'processing_metadata'? True

   document_info content:
      • document_status: acti

## 8. Advanced Queries - Query theo Status

Ví dụ các query hữu ích với status fields mới

In [11]:
print("📊 Example Queries với Status Fields:\n")

# Query 1: Chỉ lấy active documents
print("1️⃣  Get all ACTIVE documents:")
cur.execute("""
    SELECT 
        metadata->>'doc_type' as doc_type,
        COUNT(*) as count
    FROM documents
    WHERE metadata->'document_info'->>'document_status' = 'active'
    GROUP BY doc_type
    ORDER BY count DESC;
""")
active_docs = cur.fetchall()
print("   Active documents by type:")
for doc_type, count in active_docs:
    print(f"     {doc_type:15} {count:>5} docs")

# Query 2: Lấy documents theo doc_type và status
print("\n2️⃣  Get bidding documents with specific status:")
cur.execute("""
    SELECT 
        metadata->>'doc_id' as doc_id,
        metadata->>'title' as title,
        metadata->'document_info'->>'document_status' as status
    FROM documents
    WHERE metadata->>'doc_type' = 'bidding'
      AND metadata->'document_info'->>'document_status' = 'active'
    LIMIT 3;
""")
bidding_active = cur.fetchall()
for doc_id, title, status in bidding_active:
    print(f"   {doc_id}: {title[:50]}... [{status}]")

# Query 3: Lấy documents bị failed trong processing
print("\n3️⃣  Get FAILED processing documents (should be none):")
cur.execute("""
    SELECT 
        metadata->>'doc_id' as doc_id,
        metadata->'processing_metadata'->>'processing_status' as status,
        metadata->'processing_metadata'->>'error_message' as error
    FROM documents
    WHERE metadata->'processing_metadata'->>'processing_status' = 'failed'
    LIMIT 5;
""")
failed_docs = cur.fetchall()
if failed_docs:
    for doc_id, status, error in failed_docs:
        print(f"   {doc_id}: {status} - {error}")
else:
    print("   ✅ No failed documents found!")

# Query 4: Combined query - Active legal docs that were successfully processed
print("\n4️⃣  Get ACTIVE legal documents with COMPLETED processing:")
cur.execute("""
    SELECT 
        metadata->>'doc_type' as doc_type,
        COUNT(*) as count
    FROM documents
    WHERE metadata->>'doc_type' IN ('law', 'decree', 'circular', 'decision')
      AND metadata->'document_info'->>'document_status' = 'active'
      AND metadata->'processing_metadata'->>'processing_status' = 'completed'
    GROUP BY doc_type
    ORDER BY count DESC;
""")
legal_active_completed = cur.fetchall()
for doc_type, count in legal_active_completed:
    print(f"   {doc_type:15} {count:>5} docs")

📊 Example Queries với Status Fields:

1️⃣  Get all ACTIVE documents:


InterfaceError: cursor already closed

## 9. Update Specific Documents với Custom Status

Ví dụ: Update một số documents với status khác (OUTDATED, SUPERSEDED, etc.)

In [ ]:
# Example: Đánh dấu một document cũ là OUTDATED

def mark_document_as_outdated(doc_id_search: str, reason: str = "Replaced by newer version"):
    """
    Đánh dấu một document là OUTDATED dựa trên doc_id.
    
    Args:
        doc_id_search: doc_id của document cần update (e.g., '43/2022/NĐ-CP')
        reason: Lý do outdated
    """
    cur.execute("""
        SELECT id, metadata 
        FROM documents
        WHERE metadata->>'doc_id' = %s;
    """, (doc_id_search,))
    
    result = cur.fetchone()
    if not result:
        print(f"❌ Document with doc_id '{doc_id_search}' not found!")
        return False
    
    doc_id, metadata = result
    
    # Update to OUTDATED
    updated = add_document_status_to_metadata(metadata, DocumentStatus.OUTDATED)
    
    # Add reason to processing_metadata
    if 'processing_metadata' not in updated:
        updated['processing_metadata'] = {}
    updated['processing_metadata']['status_change_reason'] = reason
    updated['processing_metadata']['status_changed_at'] = datetime.now().isoformat()
    
    # Update database
    if update_document_metadata(doc_id, updated):
        print(f"✅ Marked document '{doc_id_search}' (ID: {doc_id}) as OUTDATED")
        print(f"   Reason: {reason}")
        return True
    else:
        print(f"❌ Failed to update document '{doc_id_search}'")
        return False


# Example usage (commented out - uncomment to use):
# mark_document_as_outdated('43/2022/NĐ-CP', 'Replaced by 43/2024/NĐ-CP')

print("✅ Function defined: mark_document_as_outdated()")
print("\nUsage example:")
print("  mark_document_as_outdated('43/2022/NĐ-CP', 'Replaced by 43/2024/NĐ-CP')")

## 10. Cleanup và Close Connection

In [ ]:
# Close cursor and connection
cur.close()
conn.close()

print("✅ Database connection closed")
print("\n📋 Summary:")
print("  - Đã cập nhật metadata cho tất cả documents")
print("  - Thêm document_status (tracking hiệu lực tài liệu)")
print("  - Thêm processing_status (tracking trạng thái xử lý)")
print("  - Sẵn sàng sử dụng cho queries và filtering")